In [1]:
import pandas as pd
import numpy as np

In [18]:
# IMPORTAÇÃO DA BASE DE SÓCIOS
socios_empresas_rotuladas_df = pd.read_csv(r'C:\DATASETS\Empresas Rotuladas\socios_empresas_rotuladas.csv', dtype=str)
socios_empresas_rotuladas_df

,cnpj,cnpj_cpf_socio,nome_socio,tempo_de_sociedade,cnpj_cpf_socio_mascarado
0,00000000000191,***923641**,MARCIO HAMILTON FERREIRA,11,***923641**
1,00000000000191,***491386**,NILSON MARTINIANO MOREIRA,11,***491386**
2,00000000000191,***718468**,WALTER MALIENI JUNIOR,11,***718468**
3,00000000000191,***415907**,CARLOS ALBERTO ARAUJO NETTO,9,***415907**
4,00000000000191,***022878**,ANTONIO MAURICIO MAURANO,9,***022878**
...,...,...,...,...,...
1683063,97541975000169,26090628859,EDIVALDO ALVES DE ARAUJO,10,***906288**
1683064,97542225000101,01095040707,CARLOS HENRIQUE ROCHA DE SOUZA,10,***950407**
1683065,97543869000114,72399317491,MARIO JORGE LIMA DA SILVA,10,***993174**
1683066,97548003000104,03028898330,CARLOS ADRIANO ROSA DA SILVA,10,***288983**


In [19]:
# Remover coluna de tempo_de_sociedade, não necessária aqui
socios_empresas_rotuladas_df.drop(columns=['tempo_de_sociedade'], inplace=True)

### Junção com a base de inidôneos e suspensos

In [5]:
# IMPORTAÇÃO DA BASE DE INIDÔNEOS E SUSPENSOS
inidoneas_e_suspensas_df = pd.read_csv(r'C:\\DATASETS\Empresas Inidoneas e Suspensas\\sancoes-inidoneas_e_suspensas.csv', dtype=str)
inidoneas_e_suspensas_df

,CPF_CNPJ,NOME
0,13459221000174,FLEXNAUTICA EMBARCACOES COMERCIO E SERVICOS EI...
1,01276136000191,COMERCIAL FERUMA LTDA
2,76258672000163,PEDRA AZUL SERVICOS DE LIMPEZA E CONSERVACAO LTDA
3,17015396000143,PRATO CERTO - ALIMENTACAO E NUTRICAO EIRELI
4,21545589000183,R & B PRODUTOS E SERVICOS LTDA
...,...,...
11915,53290712087,EDUARDO FLORENTINO PACHECO DA SILVA
11916,15158268134,JOSE VALDECIO PESSOA
11917,15772497472,UBIRACI BERNADINO GOMES
11918,89196988800,MARCIO EMILIO BARSANTI DE ALMEIDA


In [9]:
print(inidoneas_e_suspensas_df.CPF_CNPJ.str.len().value_counts())
pj_inid_susp_df = inidoneas_e_suspensas_df[inidoneas_e_suspensas_df.CPF_CNPJ.str.len()==14]
pf_inid_susp_df = inidoneas_e_suspensas_df[inidoneas_e_suspensas_df.CPF_CNPJ.str.len()==11]

11    6770
14    5150
Name: CPF_CNPJ, dtype: int64


In [20]:
# Rotula os sócios que são PJ
socios_empresas_rotuladas_df['socio_inidoneo_suspenso'] = (socios_empresas_rotuladas_df.merge(pj_inid_susp_df, left_on='cnpj_cpf_socio', right_on='CPF_CNPJ', how='left', indicator=True).eval('_merge == "both"').astype(bool))
socios_empresas_rotuladas_df.groupby(socios_empresas_rotuladas_df.cnpj_cpf_socio.str.len()).socio_inidoneo_suspenso.value_counts()

cnpj_cpf_socio  socio_inidoneo_suspenso
11              False                      1664526
14              False                        18516
                True                            26
Name: socio_inidoneo_suspenso, dtype: int64

In [22]:
# Rotula os sócios que são PF

# Primeiro, mascara o CPF na base de pessoas físicas inidoneas/suspensas:
pf_inid_susp_df['CPF_MASCARADO'] = np.where(pf_inid_susp_df.CPF_CNPJ.str.len()==14, pf_inid_susp_df.CPF_CNPJ, '***' + pf_inid_susp_df.CPF_CNPJ.str[3:9] + '**')

socios_empresas_rotuladas_df['socio_inidoneo_suspenso_pf'] = (socios_empresas_rotuladas_df.merge(pf_inid_susp_df, left_on=['cnpj_cpf_socio_mascarado', 'nome_socio'], right_on=['CPF_MASCARADO', 'NOME'], how='left', indicator=True).eval('_merge == "both"').astype(bool))
socios_empresas_rotuladas_df.groupby(socios_empresas_rotuladas_df.cnpj_cpf_socio.str.len()).socio_inidoneo_suspenso_pf.value_counts()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


cnpj_cpf_socio  socio_inidoneo_suspenso_pf
11              False                         1664401
                True                              125
14              False                           18542
Name: socio_inidoneo_suspenso_pf, dtype: int64

In [23]:
# Junta os marcadores:
socios_empresas_rotuladas_df['socio_inidoneo_suspenso'] = socios_empresas_rotuladas_df['socio_inidoneo_suspenso']|socios_empresas_rotuladas_df['socio_inidoneo_suspenso_pf']
socios_empresas_rotuladas_df.drop(columns=['socio_inidoneo_suspenso_pf'], inplace=True)
socios_empresas_rotuladas_df.groupby(socios_empresas_rotuladas_df.cnpj_cpf_socio.str.len()).socio_inidoneo_suspenso.value_counts()

cnpj_cpf_socio  socio_inidoneo_suspenso
11              False                      1664401
                True                           125
14              False                        18516
                True                            26
Name: socio_inidoneo_suspenso, dtype: int64

Fim do processamento de Inidôneos e Suspensos.

### Junção com a base de pessoas punidas:

In [26]:
# IMPORTAÇÃO DA BASE DE PESSOAS PUNIDAS
punidas_df = pd.read_csv(r'C:\DATASETS\Empresas Punidas\punidas.csv', dtype=str)
punidas_df

,CPF_CNPJ,NOME
0,21504073000190,CENTRO DE FORMACAO DE CONDUTORES SERRACAR LARA...
1,10699386000125,TEC LIGHT INDUSTRIA E COMERCIO DE REATORES EIRELI
2,07486627000143,C.A.M. CASTILHO ME
3,21463538000102,COMERCIAL PICAPAU EIRELI - ME
4,06940189000189,MEC PRESS ENGENHARIA DO BRASIL EIRELI
...,...,...
198,25980213805,RODRIGO ZANARDO
199,30872074897,BRUNO SOARES BERNARDO
200,14403279805,VIVALDO DIAS DA SILVA
201,03133444930,CARLOS ARTURO MALLORQUIN JUNIOR


In [27]:
print(punidas_df.CPF_CNPJ.str.len().value_counts())
pj_punidas_df = punidas_df[punidas_df.CPF_CNPJ.str.len()==14]
pf_punidas_df = punidas_df[punidas_df.CPF_CNPJ.str.len()==11]

14    192
11     11
Name: CPF_CNPJ, dtype: int64


In [28]:
# Rotula os sócios que são PJ
socios_empresas_rotuladas_df['socio_punido'] = (socios_empresas_rotuladas_df.merge(pj_punidas_df, left_on='cnpj_cpf_socio', right_on='CPF_CNPJ', how='left', indicator=True).eval('_merge == "both"').astype(bool))
socios_empresas_rotuladas_df.groupby(socios_empresas_rotuladas_df.cnpj_cpf_socio.str.len()).socio_punido.value_counts()

cnpj_cpf_socio  socio_punido
11              False           1664526
14              False             18541
                True                  1
Name: socio_punido, dtype: int64

In [29]:
# Rotula os sócios que são PF

# Primeiro, mascara o CPF na base de pessoas físicas punidas:
pf_punidas_df['CPF_MASCARADO'] = np.where(pf_punidas_df.CPF_CNPJ.str.len()==14, pf_punidas_df.CPF_CNPJ, '***' + pf_punidas_df.CPF_CNPJ.str[3:9] + '**')

socios_empresas_rotuladas_df['socio_punido_pf'] = (socios_empresas_rotuladas_df.merge(pf_punidas_df, left_on=['cnpj_cpf_socio_mascarado', 'nome_socio'], right_on=['CPF_MASCARADO', 'NOME'], how='left', indicator=True).eval('_merge == "both"').astype(bool))
socios_empresas_rotuladas_df.groupby(socios_empresas_rotuladas_df.cnpj_cpf_socio.str.len()).socio_punido_pf.value_counts()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


cnpj_cpf_socio  socio_punido_pf
11              False              1664526
14              False                18542
Name: socio_punido_pf, dtype: int64

In [31]:
# Junta os marcadores:
socios_empresas_rotuladas_df['socio_punido'] = socios_empresas_rotuladas_df['socio_punido']|socios_empresas_rotuladas_df['socio_punido_pf']
socios_empresas_rotuladas_df.drop(columns=['socio_punido_pf'], inplace=True)
socios_empresas_rotuladas_df.groupby(socios_empresas_rotuladas_df.cnpj_cpf_socio.str.len()).socio_punido.value_counts()

cnpj_cpf_socio  socio_punido
11              False           1664526
14              False             18541
                True                  1
Name: socio_punido, dtype: int64

Fim do processamento de pessoas punidas.

### Junção com a base de Entidades sem Fins Lucrativos Impedidas

In [33]:
# IMPORTAÇÃO DA BASE DE Entidades sem Fins Lucrativos Impedidas
esfl_impedidas_df = pd.read_csv(r'C:\DATASETS\Entidades sem Fins Lucrativos Impedidas\sem_fim_lucr_impedidas.csv', dtype=str)
esfl_impedidas_df

,CNPJ
0,06303088000105
1,01923684000166
2,03652447000133
3,03863259000154
4,07550679000131
...,...
2532,05062489000140
2533,01052752000169
2534,15243363000125
2535,07642296000193


In [34]:
print(esfl_impedidas_df.CNPJ.str.len().value_counts())

14    2537
Name: CNPJ, dtype: int64


In [35]:
# Rotula os sócios que são PJ (só temos esses)
socios_empresas_rotuladas_df['socio_esfl_punido'] = (socios_empresas_rotuladas_df.merge(esfl_impedidas_df, left_on='cnpj_cpf_socio', right_on='CNPJ', how='left', indicator=True).eval('_merge == "both"').astype(bool))
socios_empresas_rotuladas_df.groupby(socios_empresas_rotuladas_df.cnpj_cpf_socio.str.len()).socio_esfl_punido.value_counts()

cnpj_cpf_socio  socio_esfl_punido
11              False                1664526
14              False                  18541
                True                       1
Name: socio_esfl_punido, dtype: int64

Fim do processamento de entidades sem fim lucrativo punidas.

### Feature engineering:

##### Criação da variável 'sancionado_sancionado', com base nas 3 variáveis anteriores.

In [39]:
socios_empresas_rotuladas_df['socio_sancionado'] = socios_empresas_rotuladas_df['socio_inidoneo_suspenso']
                                                   |socios_empresas_rotuladas_df['socio_punido']
                                                   |socios_empresas_rotuladas_df['socio_esfl_punido']
socios_empresas_rotuladas_df.groupby(socios_empresas_rotuladas_df.cnpj_cpf_socio.str.len())
                            .socio_sancionado.value_counts()

cnpj_cpf_socio  socio_sancionado
11              False               1664401
                True                    125
14              False                 18514
                True                     28
Name: socio_sancionado, dtype: int64

In [41]:
# Remover colunas não necessárias
socios_empresas_rotuladas_df.drop(columns=['socio_inidoneo_suspenso', 'socio_punido', 'socio_esfl_punido'], inplace=True)

In [42]:
# Salvando resultado:
socios_empresas_rotuladas_df.to_csv(r'C:\DATASETS\Empresas Rotuladas\socios_empresas_rotuladas-socio_sancionado.csv', index = False)